# Analyzing Hacker News Posts

This is an exploratory Jupyter notebook that aims to pull insights from posts submissed to a technology site called Hacker News. The aim is to understand the optimal time to post two types of content: `Ask HN` and `Show HN`. We will determine optimal times by finding the average number of comments for each type of post at each hour in a 24-hour cycle.


<img src="https://zox.news/wp-content/uploads/2021/04/hn-dirt-big.png" alt="Kitten" title="Hacker news" 
     width='100%' />


## Overview

[Hacker News](https://news.ycombinator.com/) is a site started by the startup incubator, [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to Reddit.


## Dataset Information

The original dataset can be found [here](https://www.kaggle.com/hacker-news/hacker-news-posts). For this analysis, the data was reduced from almost *300,000 rows* to approximately *20,000 rows*. We did this by eliminating submissions that didn't receive any comments and then randomly sampling from the remaining submissions. The `7` columns in the dataset are described below:

- `id`:- Unique identifier from Hacker News for the post

- `title`:- Title of the post

- `url`:- The URL that the posts links to, if the post has a URL

- `num_points`:- Number of points the post acquired (calculated as *total upvotes - total downvotes*)

- `num_comments`:- Number of comments on the post

- `author`:- Username of the person who submitted the post

- `created_at`:- Date and time of the post's submission

## Categories of Interest

We're specifically interested in posts with titles that begin with either Ask HN or Show HN. 

- Users submit `Ask HN` posts to ask the Hacker News community a specific question, for example:

```
Ask HN: How to improve my personal website?
Ask HN: Am I the only one outraged by Twitter shutting down share counts?

```
- Users submit `Show HN` posts to show the Hacker News community a project, product, or just something interesting. Below are a few examples:

```
Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
Show HN: Something pointless I made

```

## Importing Libraries
As we progress in this project, we will need to read files, work with dates and times, and create visualizations. For these three aims, we import python's `reader` and `datetime` modules, and draw from plotly's visualisation libraries - `plotly.express`, `plotly.graph_objects` and `plotly.subplots`.

In [1]:
from csv import reader
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ModuleNotFoundError: No module named 'plotly'

In [3]:
def extract_data(filepath):
    opened_file = open('hacker_news.csv')
    read_file = csv.reader(opened_file)
    result = list(read_file)
    return result[0], result[1:]

In [4]:
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

In [5]:
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


#### Filter Posts for 'Ask' or 'Show HN'

In [13]:
ask_posts = []
show_posts = []
other_posts = []

In [14]:
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [15]:
print(f"There are {len(ask_posts)} ask posts, {len(show_posts)} show posts, and {len(other_posts)} other posts.")

There are 1744 ask posts, 1162 show posts, and 17194 other posts.


#### Identify the total number of comments in ask posts.

In [16]:
print(ask_posts[0])

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


In [17]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

In [18]:
print(f"Total number of comments: {total_ask_comments}")
print(f"Average number of comments: {total_ask_comments / len(ask_posts)}")


Total number of comments: 24483
Average number of comments: 14.038417431192661


#### Identify the total number of comments in show posts.

In [19]:
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

In [20]:
print(f"Total number of comments: {total_show_comments}")
print(f"Average number of comments: {total_show_comments / len(show_posts)}")

Total number of comments: 11988
Average number of comments: 10.31669535283993


It appears that 'Ask' comments tend to receive more comments on average.

#### What post time is optimal to receive the most comments on 'Ask posts'?

In [32]:
import datetime as dt

In [33]:
print(ask_posts[0])

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


In [34]:
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

In [35]:
result_list[0]

['8/16/2016 9:55', 6]

In [36]:
counts_by_hour = {}
comments_by_hour = {}

In [37]:
for row in result_list:
    time = row[0]
    num_comments = row[1]
    date, hour = time.split()
    hour_dt = dt.datetime.strptime(hour, '%H:%M')
    final_hour = hour_dt.strftime('%H')
    
    if final_hour not in counts_by_hour:
        counts_by_hour[final_hour] = 1
        comments_by_hour[final_hour] = num_comments
    else:
        counts_by_hour[final_hour] += 1
        comments_by_hour[final_hour] += num_comments

In [38]:
print(counts_by_hour)

{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


In [39]:
print(comments_by_hour)

{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}


Calculate the average number of comments, per post, for each hour of the day.

In [53]:
def compute_average(dict_a, dict_b):
    result = []
    
    for key in dict_a:
        average = dict_b[key] / dict_a[key]
        result.append([key, average])
    return result

In [58]:
avg_by_hour = compute_average(counts_by_hour, comments_by_hour)

In [60]:
avg_by_hour

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

### Top 5 Hours for Ask Posts Comments

In [76]:
swap_avg_by_hour = []
for row in avg_by_hour:
    time = row[0]
    count = row[1]
    swap_avg_by_hour.append([count, time])
print(swap_avg_by_hour)

[[5.5777777777777775, '09'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [16.796296296296298, '16'], [7.985294117647059, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.796296296296297, '03'], [10.08695652173913, '05'], [10.8, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.170212765957447, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


In [77]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [78]:
for row in sorted_swap[:5]:
    hour = row[0]
    count = row[1]
    print(f"{hour}: {count:.2f} average comments per post")

ValueError: Unknown format code 'f' for object of type 'str'